In [1]:
#imports
import numpy as np, pandas as pd, os, openpyxl, re

In [2]:
with open("expspd.txt","r") as f:
    txt = list(filter(lambda x:not re.match(r"\s*$",x), f.read().split("\n-------end session-------\n")))

Runtime was capped at 2000ms and 5 trials of every setting done and averaged out.

In [3]:
#initializing each necessary dataframe
df1k = pd.DataFrame(columns=['nthreads','Serial','LockFree','TASLock','pthread_mutex','ALock','CLHLock'])
df1k.loc[:,'nthreads'] = [2, 3, 4, 8, 14, 28]
df1k.set_index('nthreads',inplace=True)
df1k.loc[:,:] = np.zeros((6,6))

df2k, df4k, df8k = [df1k.copy() for _ in range(3)]

In [4]:
typedict = {-2:'Serial', -1:'LockFree', 0:'TASLock', 1:'pthread_mutex', 2:'ALock', 3:'CLHLock'}
workdict = {1000:df1k,2000:df2k,4000:df4k,8000:df8k}

In [5]:
#parse
for s in range(len(txt)):
    W = int(re.search(r'(?<=pkwk: )[0-9]+',txt[s]).group())
    nthreads = int(re.search(r'(?<=nsrc: )[0-9]+',txt[s]).group()) + 1
    processed = int(re.search(r"(?<=processed: )[0-9]+",txt[s]).group())
    strat = int(re.search(r'(?<=opmode: )\-*[0-9]+',txt[s]).group())
    locktype = int(re.search(r'(?<=lockmode: )\-*[0-9]+',txt[s]).group())
    if strat==-1 and locktype==-1:
        locktype = -2
    workdict[W].loc[nthreads,typedict[locktype]] += processed

In [6]:
#turn into avg throughput ratio to serial, write to excel
with pd.ExcelWriter('expspdEX.xlsx') as writer:
    for key in workdict.keys():
        workdict[key] /= 5*2000
        for j in range(1,6):
            workdict[key].iloc[:,j] /= workdict[key].iloc[:,0]
        workdict[key].to_excel(writer,sheet_name="W={}".format(key))

In [7]:
df1k

,Serial,LockFree,TASLock,pthread_mutex,ALock,CLHLock
nthreads,,,,,,
2,387.5892,0.930476,0.922080,0.916089,0.926078,0.927233
3,290.5922,0.497141,0.494787,0.482365,0.495006,0.495545
4,322.0651,0.881385,0.878870,0.868326,0.878588,0.872170
8,371.6076,1.263812,1.246361,1.244384,1.241004,1.239886
14,384.4981,2.809045,2.760091,2.694967,2.513844,2.543087
28,398.6686,0.019094,0.017796,0.016908,0.018195,0.017959


In [8]:
df2k

,Serial,LockFree,TASLock,pthread_mutex,ALock,CLHLock
nthreads,,,,,,
2,205.7319,0.947139,0.941939,0.938922,0.941765,0.942965
3,148.2632,0.497878,0.485824,0.484468,0.497118,0.496334
4,164.8873,0.883097,0.882047,0.872915,0.880466,0.877547
8,190.9406,1.235988,1.246619,1.237527,1.233704,1.232967
14,202.8261,2.571520,2.666893,2.620300,2.644971,2.519311
28,204.9781,0.034601,0.035014,0.032871,0.034913,0.035508


In [9]:
df4k

,Serial,LockFree,TASLock,pthread_mutex,ALock,CLHLock
nthreads,,,,,,
2,104.4963,0.971190,0.968774,0.967476,0.970477,0.970720
3,75.0794,0.498821,0.485555,0.498027,0.498055,0.496959
4,83.4870,0.883941,0.884262,0.880266,0.883230,0.881790
8,96.6151,1.248438,1.249251,1.228060,1.230959,1.245862
14,102.9090,2.628760,2.654970,2.632629,2.568604,2.645405
28,104.0531,0.068061,0.069304,0.068127,0.069146,0.069048


In [10]:
df8k

,Serial,LockFree,TASLock,pthread_mutex,ALock,CLHLock
nthreads,,,,,,
2,52.6281,0.987651,0.985911,0.984090,0.985475,0.985800
3,37.5828,0.498765,0.497986,0.499111,0.498994,0.497930
4,41.9452,0.882296,0.883329,0.881274,0.883207,0.880871
8,48.5050,1.240402,1.242076,1.227090,1.235679,1.235549
14,51.6564,2.609152,2.622362,2.630298,2.618411,2.554284
28,52.3460,0.133437,0.130086,0.132616,0.141289,0.149811
